## Gemini Pro 복수 호출 처리( Async, Parallel )

Feedback : shins777@gmail.com. 

* 이 Colab 은 Generative AI 사용시 복수개의 호출을 위한 Async call 과 Pralllel call 을 구현한 예제입니다. 
* 코드는 Langchain을 기반으로 처리하며 그에 따른 API는 아래 링크 참고하세요.  
    * 자세한 정보는 [README.md](https://github.com/shins777/google_gen_ai_sample/blob/main/notebook/gemini/README.md) 파일 참고하세요.

### 라이브러리 설치

In [ ]:
%pip install --upgrade --quiet langchain langchain-core langchain-google-vertexai

In [ ]:
from IPython.display import display, Markdown

### GCP 사용자 인증 / 환경설정

GCP 인증방법은 아래와 URL 정보를 참고하여 GCP에 접근 하는 환경을 구성해야 합니다. 
* https://cloud.google.com/docs/authentication?hl=ko
* 자세한 정보는 [README.md](https://github.com/shins777/google_gen_ai_sample/blob/main/notebook/gemini/README.md) 파일 참고하세요.

In [ ]:
#  아래 코드는 Colab 환경에서만 실행해주세요. 다른 환경에서는 동작하지 않습니다.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

### GCP 프로젝트 및 리전 설정
본인의 GCP 환경에 맞게 아래 설정을 구성하세요.  
* 구글의 최신버전인 gemini pro 사용을 권고드립니다.   
* 만일, 기본 버전 text bison 을 사용하려한다면, 참조하는 class 가 다르므로 주의하세요.  
* 현재 Gemini는 한국리전(asia-northeast3)을 통해서 접근이 가능합니다.

In [ ]:
model_name="gemini-pro"
project="ai-hangsik"
location="asia-northeast3"

### Responsible AI

구글은 아래와 같이 생성형 AI 처리시 Responsible AI 부분으로 Harmcategory 에 따른 LLM 응답 조절이 가능합니다.  
구글은 구글의 모델을 사용하는 사용자가 생성된 컨텐츠때문에 피해가 되지 않도록 하기 위해서 Responsible AI라는 내용으로 구글의 모델을 사용하는 사용자를 보고하고 있습니다.
이 부분은 특별한 문제가 없다면 생략해도 됩니다.

Responsible AI 설정을 위한 클래스입니다.  
*   HarmCategory : https://cloud.google.com/vertex-ai/docs/reference/rest/v1/HarmCategory
*   HarmBlockThreshold : https://cloud.google.com/php/docs/reference/cloud-ai-platform/0.31.0/V1.SafetySetting.HarmBlockThreshold

In [ ]:
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory

safety_settings = {
                    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
                    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
                    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE
}

### 모델 초기화 및 기본 실행

아래 내용은 Langchain 기반에서의 gemini_pro 를 초기화하는 방법입니다.  
업무에 따라서 Langchain 을 사용하셔도 좋고, 만일 latency 가 중요한 업무이면 Google native API를 사용하는것도 Latency를 줄이는 방법이 될수 있습니다.

* Langchain VertexAI API : https://api.python.langchain.com/en/stable/llms/langchain_google_vertexai.llms.VertexAI.html#langchain_google_vertexai.llms.VertexAI
* API 참고(Python SDK) : https://cloud.google.com/vertex-ai/generative-ai/docs/sdk-for-llm/llm-sdk-overview
* API 참고(REST API) : https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/overview?hl=ko

Vertex AI에 대한 Langchain 라이브러리를 보시려면 아래 URL을 참고하세요.
* https://python.langchain.com/docs/integrations/llms/google_vertex_ai_palm

참고!! : Langchain 에서 Google AI, GoogleGenerativeAI 또는 langchain-google-genai 로 표시된 API는 Consumer 버전의 API이므로 Enteprise(기업)용 사용자는 반드시 위의 Vertex AI 용으로 사용해주세요.

In [ ]:
from langchain_google_vertexai.llms import VertexAI

gemini_pro = VertexAI( model_name=model_name,
                  project=project,
                  location=location,
                  verbose=True,
                  streaming=True,
                  safety_settings = safety_settings,
                  temperature = 0.2,
                  top_p = 1,
                  top_k = 10
                 )

### Asynch Call
비즈니스 로직상에서 Async call 이 필요한 요건에 활용할 수 있니다. 필요에 따라서 배치형태의 호출에도 사용될 수 있습니다.
Async 호출은 Latency를 줄이고, 아키텍처 유연성을 확보하기 위해서 필요한 호출 방식입니다.

* Langchain Reference : https://api.python.langchain.com/en/stable/llms/langchain_google_vertexai.llms.VertexAI.html#langchain_google_vertexai.llms.VertexAI.agenerate
    * VertexA class 내의 함수중 async 가 표시된 함수들을 참고하세요. (ex> async agenerate )


#### 호출에 필요한 함수 정의

In [ ]:
import time
import asyncio
import threading

# 현재 일반 사용자 Colab에서 2 core 밖에 없어서 Aysnc 처리시 아래 코드 필요.
# https://stackoverflow.com/questions/55409641/asyncio-run-cannot-be-called-from-a-running-event-loop-when-using-jupyter-no
import nest_asyncio
nest_asyncio.apply()

questions = ["한국의 수도는 어디인가요 ?","일본의 수도는 어디인가요 ?","미국의 수도는 어디인가요 ?","영국의 수도는 어디인가요 ?","프랑스의 수도는 어디인가요 ?"]

def synch_generate(llm: VertexAI, prompt:str):
  """
  LLM Sync Call를 위한 함수 
  llm.generate 
  """
  resp = llm.generate(prompts =[prompt])
  print(f"Thread:{threading.get_ident()} : {resp.generations[0][0].text}")

def synch_call(llm: VertexAI):  
  result = [synch_generate(llm, questions[i]) for i in range(5)]

#------------------------------------------------

# Define an async function.
async def async_generate(llm:VertexAI, prompt:str):
  """
  LLM Sync Call를 위한 함수 
  llm.agenerate 
  """  
  resp = await llm.agenerate(prompts =[prompt])
  print(f"Thread:{threading.get_ident()} : {resp.generations[0][0].text}")

async def generate_concurrently(llm:VertexAI):
  tasks = [async_generate(llm, questions[i] ) for i in range(5)]
  await asyncio.gather(*tasks)

#### Sync, Async 호출 비교
아래 코드를 통해서 Sync call을 통한 단계적인 호출보다 aync call를 사용한 경우 같은 thread를 사용하더라도 Latency에서 이점을 얻을 수 있습니다.

In [ ]:

start = time.perf_counter()
synch_call(gemini_pro)
elapsed = time.perf_counter() - start
print(f"Serially executed in {elapsed:0.2f} seconds.\n")

start = time.perf_counter()
asyncio.run(generate_concurrently(gemini_pro))
elapsed = time.perf_counter() - start
print(f"Concurrently executed in {elapsed:0.2f} seconds.\n")

### Concurrent call ( Multiprocessing )
* Async call과 더불어 동시처리에 자주 활용되는 Paralllel call에 대한 예제입니다.
* Parallel call 은 여러개의 호출을 Multi thread 기반에서 호출함으로써 CPU core 수에 따라서 대량 배치처리에 대한 Latency를 줄일수 있습니다.

#### 호출에 사용되는 함수 정의
Parallel 처리시 차이점을 확인하는것과 함께, Lanchain을 활용할 때와 Gemini Pro 자체 API를 활용할 때와의 Latency도 비교하기 위해서 두가지 함수를 사용합니다.

In [ ]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models

vertexai.init(project="ai-hangsik", location="us-central1")

def langchain_call(country:str)->str:
  """
  Langchain 기반의 Gemini Pro call
  """
  capital = gemini_pro.invoke(f"{country} 의 수도는 ?")

  print(f"[langchain_call] Thread: {threading.get_ident()} arg:{country}  return : {capital}")
  return capital

def native_call(country:str)-> str:
  """
  Gemini Pro 자체 API 기반의call
  """  
  model = GenerativeModel("gemini-1.0-pro-001")
  responses = model.generate_content(
    [f"{country} 의 수도는 ?"],
  )
  capital = responses.text

  print(f"[native_call] Thread: {threading.get_ident()} arg:{country}  return : {capital}")
  return capital

#### Sequencial call
* 아래 코드는 순차적으로 호출할때의 latency를 측정하기 위한 목적입니다.
* Lanchain API, Gemini Pro API 를 사용할때 각각을 측정합니다.

In [ ]:
import time

#----[ Lanchain Call ]-----
t1 = time.time()
langchain_call("대한민국")
langchain_call("일본")
langchain_call("중국")
t2 = time.time()

print(f"Langchain API call : {t2-t1}\n")

#----[ Native Call ]-----
t1 = time.time()
native_call("대한민국")
native_call("일본")
native_call("중국")
t2 = time.time()

print(f"Gemini Pro API call : {t2-t1}\n")

#### ThreadPoolExecutor - map function
* 아래 코드는 threadpool을 활용한 multi 호출을 나타내는 예제입니다.
* 처리 결과를 보면 다른 쓰레드에 의해서 각각의 Tasks 가 실행된 것을 알수 있습니다. 
* map 을 사용하여 처리하였으며, map을 arg 와 return 값을 순서처리를 보장하지는 않습니다. 
    * https://docs.python.org/3/library/concurrent.futures.html

In [ ]:
import threading
from concurrent.futures import ThreadPoolExecutor

args_list ={"대한민국", "일본", "중국"}

#----[ Lanchain Call ]-----
t1 = time.time()

with ThreadPoolExecutor(max_workers=10) as executor:
    results = executor.map(langchain_call, args_list)

print(','.join(results))

t2 = time.time()

print(f"Langchain API call : {t2-t1}\n")

#----[ Native Call ]-----

t1 = time.time()

with ThreadPoolExecutor(max_workers=10) as executor:
    results = executor.map(native_call, args_list)

print(','.join(results))

t2 = time.time()

print(f"Gemini Pro API call : {t2-t1}\n")

#### ThreadPoolExecutor - submit function
* 아래 코드는 threadpool을 활용한 multi 호출을 나타내는 예제입니다.
* 처리 결과를 보면 다른 쓰레드에 의해서 각각의 Tasks 가 실행된 것을 알수 있습니다. 
* submit 을 사용하여 처리하였으며, submit을 사용하면 arg 에 대한 리턴값에 대한 순서를 보장할수 있습닞다.
    * https://docs.python.org/3/library/concurrent.futures.html
    * 호출된 순서와 리턴된 값의 순서가 동일함.
    * 이 부분은 Concurrent 처리시 참고필요.

* 동시처리를 위한 Linux core 수 확인.  
* Cloud shell container 기준 : 4 cores. 
* Colab container 기준 : 2 cores

In [ ]:
!nproc

In [ ]:
import time
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures

args_list ={"대한민국", "일본", "중국"}

#----[ Lanchain Call ]-----

t1 = time.time()

with ThreadPoolExecutor(max_workers=10) as executor:

    futures = [executor.submit(langchain_call, arg) for arg in args_list]
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

print(results)

t2 = time.time()

print(f"Langchain API call : {t2-t1}\n")

#----[ Native Call ]-----

t1 = time.time()

args_list ={"대한민국", "일본", "중국"}

with ThreadPoolExecutor(max_workers=10) as executor:

    futures = [executor.submit(native_call, arg) for arg in args_list]
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

print(results)

t2 = time.time()

print(f"Gemini Pro API call : {t2-t1}\n")

